In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.callbacks import ModelCheckpoint
import os.path
from keras.preprocessing.image import ImageDataGenerator

def fixed_generator(generator):
    for batch in generator:
        yield (batch, batch)

# dimensions of our images.
img_width, img_height = 256, 256

train_data_dir = '/nfs/home/pgulyaev/train'
validation_data_dir = '/nfs/home/pgulyaev/valid'
nb_train_samples = 1514
nb_validation_samples = 283
epochs = 50
batch_size = 20

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
input_img = Input(shape=input_shape)

# Block 1
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_img)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), name='block1_pool')(x)

# Block 2
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = MaxPooling2D((2, 2), name='block2_pool')(x)

# Block 3
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = MaxPooling2D((2, 2), name='block3_pool')(x)

# Block 4
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = MaxPooling2D((2, 2), name='block4_pool')(x)

# Block 5
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
encoded = MaxPooling2D((2, 2), name='block5_pool')(x)

x = UpSampling2D((2, 2))(encoded)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block5_conv3')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block5_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block5_conv1')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block4_conv3')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block4_conv2')(x)
x = Conv2D(512, (3, 3), activation='relu', padding='same', name='decode_block4_conv1')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decode_block3_conv3')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decode_block3_conv2')(x)
x = Conv2D(256, (3, 3), activation='relu', padding='same', name='decode_block3_conv1')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decode_block2_conv2')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same', name='decode_block2_conv1')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decode_block1_conv2')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same', name='decode_block1_conv1')(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
autoencoder = Model(input_img, decoded)
weights_file="/nfs/home/pgulyaev/cae_vgg16.best.hdf5"
if (os.path.isfile(weights_file)):
    autoencoder.load_weights(weights_file)
autoencoder.compile(optimizer='adadelta', loss='mse')
checkpoint = ModelCheckpoint(weights_file, verbose=1, save_best_only=True, monitor='val_loss', mode='min')
callbacks_list = [checkpoint]

train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None)
import sys
oldStdout = sys.stdout
fileLog = open('/nfs/home/pgulyaev/cae_vgg16_logFile', 'w')
sys.stdout = fileLog
autoencoder.fit_generator(
        fixed_generator(train_generator),
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=fixed_generator(validation_generator),
        validation_steps=nb_validation_samples // batch_size,
        callbacks=callbacks_list)
sys.stdout = oldStdout